In [1]:
import torch
from tqdm import tqdm

device = 'cpu'

### Dataset

In [2]:
def generate_dataset(pe, n_samples):
    # Invariant features ~ N(0, I)
    X_inv = torch.randn(n_samples, 2)

    # Label Y = x1 + x2 + noise
    noise = torch.randn(n_samples) * (0.1 ** 0.5)
    Y = X_inv.sum(dim=1) + noise  # Shape: (n_samples,)

    # Spurious features ~ N([Y, Y], pe * I)
    mean_spurious = torch.stack([Y, Y], dim=1)
    spurious_noise = torch.randn(n_samples, 2) * (pe ** 0.5)
    X_env = mean_spurious + spurious_noise

    # Concatenate invariant and spurious features
    X = torch.cat([X_inv, X_env], dim=1)
    # Reshape labels to (n_samples, 1)
    Y = Y.unsqueeze(1)
    return X, Y

def combine_datasets(datasets):
    X = torch.concat([dataset[0] for dataset in datasets])
    Y = torch.concat([dataset[1] for dataset in datasets])
    return X, Y

In [3]:
d_1 = generate_dataset(0.5, 4)
d_2 = generate_dataset(1.0, 4)
d_3 = generate_dataset(9.9, 4)
comb = combine_datasets([d_1, d_2, d_3])
d_1

(tensor([[-0.5295,  1.2863,  0.5123, -0.1659],
         [-0.4916,  0.2065,  0.1590, -0.6059],
         [-0.7471, -1.0425, -1.8465, -3.5519],
         [-2.2299, -0.2309, -3.5473, -4.0008]]),
 tensor([[ 0.7578],
         [ 0.0473],
         [-2.0115],
         [-2.6922]]))

### Models

In [4]:
from torch import nn
from torch.nn import functional as F

out_features = 1

In [5]:
class FeatureExtractor(nn.Module):

    def __init__(self) -> None:
        super(FeatureExtractor, self).__init__()
        self.lin = nn.Linear(4,out_features,bias=False)

    def forward(self, x):
        return self.lin(x)

In [6]:
class Classifier(nn.Module):

    def __init__(self, mean, std, epsilon) -> None:
        super(Classifier, self).__init__()
        self.w = mean + std*epsilon

    def forward(self, x):
        return torch.matmul(x, self.w.T)

In [7]:
class AutoEncoder(nn.Module):

    def __init__(self) -> None:
        super(AutoEncoder, self).__init__()
        self.m_u = torch.nn.parameter.Parameter(torch.rand(1,out_features))
        self.std = torch.nn.parameter.Parameter(torch.rand(1,out_features))
        torch.nn.init.uniform_(self.m_u, -1, 1)
        torch.nn.init.uniform_(self.std, 0, 1)

    def recon_loss(self, X, Y, classifier, f_e): #minimize this
        return F.mse_loss(Y, classifier(f_e(X)))

    def KL_loss(self):   #maximize this
        return torch.sum(1 + torch.log(torch.square(self.std)) - torch.square(self.m_u) - torch.square(self.std))/2

    def sample(self, epsilon=None):
        if epsilon is None:
            epsilon =  torch.randn_like(self.m_u).to(device)
        return Classifier(self.m_u, self.std, epsilon)

    def fit(self, X, Y, f_e, epochs):
        optim = torch.optim.Adam([self.m_u, self.std], betas=(0.5, 0.5))
        for _ in range(epochs):
            loss = self.recon_loss(X, Y, self.sample(), f_e) - self.KL_loss()
            optim.zero_grad()
            loss.backward()
            optim.step()

### Train

In [22]:
pe_train = [0.1, 0.3, 0.5, 0.7, 0.9]
pe_val = [0.4, 0.8]
pe_test = [10, 100]
n_samples = 500

train_env = [generate_dataset(pe, n_samples) for pe in pe_train]
train_env.append(combine_datasets(train_env))
val_env = [generate_dataset(pe, n_samples) for pe in pe_val]
X_val = torch.cat([x for x, _ in val_env], dim=0)
Y_val = torch.cat([y for _, y in val_env], dim=0)
val_env = [(X_val, Y_val)]
test_env = [generate_dataset(pe, n_samples) for pe in pe_test]
X_test = torch.cat([x for x, _ in test_env], dim=0)
Y_test = torch.cat([y for _, y in test_env], dim=0)
test_env = [(X_test, Y_test)]

In [35]:
f_e = FeatureExtractor().to(device)
q_u = [AutoEncoder().to(device) for _ in train_env]

lamda = 5
with tqdm(range(1000)) as tepoch:
    for ep in tepoch:
        # Update posteriors distribution of classifiers for each training environment
        for q, D in zip(q_u, train_env):
            q.fit(D[0], D[1], f_e, 20)

        # Update feature extractor only
        loss = 0
        optim = torch.optim.Adam(f_e.parameters(), betas=(0.5, 0.5), lr=0.01)
        for _ in range(10):
            epsilon = torch.randn_like(q_u[-1].m_u).to(device)
            [ loss := loss + (1+lamda)*q_u[-1].recon_loss(D[0], D[1], q_u[-1].sample(epsilon), f_e)-lamda*q.recon_loss(D[0], D[1], q.sample(epsilon), f_e)  for q, D in zip(q_u[:-1], train_env[:-1]) ]
        loss = loss/10
        optim.zero_grad()
        loss.backward()
        optim.step()

        epsilon = torch.zeros_like(q_u[-1].m_u).to(device)
        test_acc = q_u[-1].recon_loss(test_env[0][0], test_env[0][1], q_u[-1].sample(epsilon), f_e)
        train_acc = q_u[-1].recon_loss(train_env[-1][0], train_env[-1][1], q_u[-1].sample(epsilon), f_e)
        tepoch.set_postfix_str(f'train:{train_acc}, test:{test_acc}')

        if ep%50==0:
            print(torch.matmul(q_u[-1].m_u, f_e.lin.weight))
            print(f_e.lin.weight)

  0%|          | 2/1000 [00:00<02:40,  6.22it/s, train:2.7753427028656006, test:3.357396125793457]

tensor([[-0.1106,  0.0435, -0.0517, -0.0903]], grad_fn=<MmBackward0>)
Parameter containing:
tensor([[-0.4330,  0.1702, -0.2025, -0.3537]], requires_grad=True)


  5%|▌         | 52/1000 [00:07<02:16,  6.96it/s, train:1.4322160482406616, test:1.6862764358520508]

tensor([[0.0146, 0.1525, 0.0614, 0.0144]], grad_fn=<MmBackward0>)
Parameter containing:
tensor([[0.0470, 0.4902, 0.1975, 0.0463]], requires_grad=True)


 10%|█         | 102/1000 [00:14<02:01,  7.39it/s, train:0.6221087574958801, test:1.4772703647613525]

...

 90%|█████████ | 902/1000 [02:05<00:13,  7.23it/s, train:0.25676387548446655, test:1.0496997833251953]

tensor([[0.5283, 0.5314, 0.0942, 0.1027]], grad_fn=<MmBackward0>)
Parameter containing:
tensor([[0.5470, 0.5502, 0.0975, 0.1063]], requires_grad=True)


 95%|█████████▌| 952/1000 [02:13<00:07,  6.76it/s, train:0.22807025909423828, test:1.3618232011795044]

tensor([[0.4993, 0.5024, 0.0924, 0.1007]], grad_fn=<MmBackward0>)
Parameter containing:
tensor([[0.5270, 0.5302, 0.0975, 0.1063]], requires_grad=True)


100%|██████████| 1000/1000 [02:20<00:00,  7.14it/s, train:0.1296904981136322, test:2.4738852977752686]


In [29]:
f_e.lin.weight, q_u[-1].m_u

(Parameter containing:
 tensor([[-0.5433, -0.4727, -0.0763, -0.1268]], requires_grad=True),
 Parameter containing:
 tensor([[-0.8624]], requires_grad=True))